## Sign language recognition using hand pose estimation of MediaPipe and CNN-RNN
Architettura ibrida CNN-RNN utilizzata per riconoscere il linguaggio dei segni, combinata con l'approccio hand pose estimation della libreria MediaPipe di Google. Il dataset utilizzato nello specifico è quello relativo all'alfabeto della lingua dei segni americana (ASL)

In [ ]:
#caricamento del dataset
import cv2
import glob
import os
import numpy as np

train_dir = "./SigNN Character Database"
unique_labels = sorted(os.listdir(train_dir)) # labels ordinate
print(unique_labels)

### Estrazione dei punti chiave della mano con MediaPipe
Vengono caricate tutte le immagini del dataset e vengono estratti i punti chiave della mano

In [ ]:
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# For static images:
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5)

images = []
labels = []
for idx, label in enumerate(unique_labels):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    print("Loading " + label)
    for file in os.listdir(train_dir + "/" + label):
        filepath = train_dir + "/" + label + "/" + file
        image = cv2.imread(filepath)               # images NOT flipped
        #image = cv2.flip(cv2.imread(filepath), 1) # images flipped
        # Convert the BGR image to RGB before processing.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Print handedness and draw hand landmarks on the image.
        #print('Handedness:', results.multi_handedness)
        if not results.multi_hand_landmarks:
            continue
        image_hight, image_width, _ = image.shape
        #annotated_image = image.copy()
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks_xyz = []
            for i in range(21):
                landmark = hand_landmarks.landmark[i]
                landmarks_xyz.append(landmark.x)
                landmarks_xyz.append(landmark.y)
                landmarks_xyz.append(landmark.z)  
            images.append(np.array(landmarks_xyz))
            labels.append(idx)
            
images = np.array(images)
labels = np.array(labels)

hands.close()

# Save data
np.save("images.npy", images)
np.save("labels.npy", labels)



In [ ]:
# funzione per caricare i dati estratti da MediaPipe senza dover ripetere il processo di classificazione
images = np.load("./images.npy")
labels = np.load("./labels.npy")

In [ ]:
images

In [ ]:
labels

In [ ]:
labels.shape,images.shape

In [ ]:
#serie di import 
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Reshape,Permute
from keras.layers import Conv2D, MaxPooling2D,SimpleRNN,LSTM
from keras import backend as K


### Creazione di train e test set

In [ ]:
from sklearn.model_selection import train_test_split
import keras

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, stratify = labels)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

### Costruzione del modello
Una CNN combinata con una RNN

In [ ]:
from tensorflow import keras

In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
X_train = X_train.reshape(-1, 63, 1, 1)

In [ ]:
X_test = X_test.reshape(-1, 63, 1, 1)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
batch_size = 128
num_classes = 26
epochs = 3000

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),padding='same', activation='relu', input_shape=(63,1,1)))
model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
model.add(Dropout(0.25))
# Reshape with the first argument being the number of filter in your last conv layer
model.add(Reshape((64, -1)))
model.add(Permute((2, 1)))
# può essere anche una LSTM
model.add(SimpleRNN(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

### Addestramento del modello

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history=model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

In [ ]:
model.summary()

### Valutazione del modello

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and Validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','validation'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training and Validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
y_pred_b_model = model.predict(X_test)
result = model.evaluate(X_test, y_test)
print(result)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, mean_squared_error, r2_score
import itertools
import numpy as np
from sklearn import linear_model

#Confusion matrix
#Helper function to plot confusion matrix
def plot_confusion_matrix(y, y_pred, title, normalize=False):
    if y.ndim>1:
        y = np.argmax(y, axis = 1)
        y_pred = np.argmax(y_pred, axis = 1)
    cm = confusion_matrix(y, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize = (24, 20))
    ax = plt.subplot()
    plt.imshow(cm, interpolation = 'nearest', cmap = plt.cm.Purples)
    plt.colorbar()
    plt.title("Confusion Matrix")
    tick_marks = np.arange(len(unique_labels))
    plt.xticks(tick_marks, unique_labels, rotation=45)
    plt.yticks(tick_marks, unique_labels)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    ax.title.set_fontsize(20)
    ax.xaxis.label.set_fontsize(16)
    ax.yaxis.label.set_fontsize(16)
    ax.set(title = title)
    limit = cm.max() / 2.
    fmt = '.2f' if normalize else 'd'
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if cm[i,j] < 0.01:
            plt.text(j, i, format(0, 'd'), horizontalalignment = "center",color = "black")
        else:
            plt.text(j, i, format(cm[i, j], fmt), horizontalalignment = "center",color = "white" if cm[i, j] > limit else "black")        
    
    plt.show()
    
        

#plot_confusion_matrix(y_test, y_pred_svm_model, title='Confusion matrix For SVM Model (24)', normalize=True)
plot_confusion_matrix(y_test, y_pred_b_model, title='Confusion matrix for b_model', normalize=True)